# Topic Modeling and Visualization

Topic modeling using `sklearn` and `pyLDAvis` to visualize topic modeling that works with the results of both`sklearn` and `gensim`.

 <a id="sec1"></a>
 ## 1. Prepare the notebook materials

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

Now install `pyLDAvis`:

Import the needed packages and enable pyLDAvis to work inside the notebook:

In [7]:
import pyLDAvis
import pyLDAvis.lda_model
pyLDAvis.enable_notebook()

In [8]:
import pandas

### Load city article dataset

In [10]:
accra = pandas.read_excel("LexisUni_articles/processed/accra_articles.xlsx")
 
# displaying the contents of the CSV file
accra.head()

,full_article,date
0,\n\n\n\r\nAccra Technical University to offer ...,2018-12-08
1,\n\n\n\r\nMeteorological Agency warns of rains...,2021-12-25
2,\n\n\n\r\nMining; First Ecowas Mining and Petr...,2016-01-25
3,\n\n\n\r\nMövenpick Ambassador Hotel Accra: Po...,2019-12-19
4,\n\n\n\r\nPHOTOS: Accra Floods After Heavy Dow...,2016-07-26


In [11]:
docs_raw = accra['full_article'].tolist()
print(len(docs_raw))

579


<a id="sec2"></a>
## 2. Convert to document-term matrices

Next, the raw documents are converted into document-term matrix (dtm), initially as raw counts (with `CountVectorizer`) and then as TF-IDF values (with `TfidfVectorizer`).

We're creating two different vectorizers, to notice the difference that different feature representations make in the process of modeling.

In [12]:
# Create the TF vector represetnation, this only counts the terms in each document

tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)

dtm_tf = tf_vectorizer.fit_transform(docs_raw)
print(dtm_tf.shape)

(579, 2679)


Let's look at some of the features (i.e. words in the corpus):

In [14]:
tf_vectorizer.get_feature_names_out()[500:520]

array(['construct', 'constructed', 'construction', 'consultant',
       'consumers', 'consumption', 'contact', 'contain', 'contained',
       'containers', 'containing', 'contemporary', 'content',
       'contentservices', 'context', 'continent', 'continental',
       'continue', 'continued', 'continues'], dtype=object)

To be able to compare the results of TF and TFIDF representation, we will use the same parameters.
This is why we will initialize the `TfidfVectorizer` with the parmeters of the `CountVectorizer`.

Initialize and prepare Document-Term-Matrix:

In [15]:
# TF IDF = measures term importance relative to the other documents
tfidf_vectorizer = TfidfVectorizer(**tf_vectorizer.get_params())

dtm_tfidf = tfidf_vectorizer.fit_transform(docs_raw)
print(dtm_tfidf.shape)

(579, 2679)


/Users/annalieb/miniconda3/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:2078: UserWarning: Only (<class 'numpy.float64'>, <class 'numpy.float32'>, <class 'numpy.float16'>) 'dtype' should be used. <class 'numpy.int64'> 'dtype' will be converted to np.float64.
  warnings.warn(


<a id="sec3"></a>
## 3. Fit Latent Dirichlet Allocation models

We will fit two LDA models, each with 3 topics. 

In [36]:
# for TF DTM
lda_tf = LatentDirichletAllocation(n_components=4, random_state=0)
lda_tf.fit(dtm_tf)

lda_tf

LatentDirichletAllocation(n_components=4, random_state=0)

In [31]:
# for TFIDF DTM
lda_tfidf = LatentDirichletAllocation(n_components=3, random_state=0)
lda_tfidf.fit(dtm_tfidf)

LatentDirichletAllocation(n_components=3, random_state=0)

Given that the models are probability distributions over topics and words, we will focus on their visualization to learn more about them.

<a id="sec4"></a>
## 4. Visualizing the models with pyLDAvis

Note on relevance metric: lambda influences the order in which the words are sorted. Adjust lambda to help with interpretation of the different categories. 

see this link: https://we1s.ucsb.edu/research/we1s-tools-and-software/topic-model-observatory/tmo-guide/tmo-guide-pyldavis/

In [37]:
# the TF representation model

pyLDAvis.lda_model.prepare(lda_tf, dtm_tf, tf_vectorizer)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.141562 -0.003942       1        1  38.421604
2     -0.150017 -0.064692       2        1  22.336259
1      0.051189 -0.066158       3        1  21.928952
3     -0.042734  0.134792       4        1  17.313185, topic_info=             Term        Freq       Total Category  logprob  loglift
388          city  555.000000  555.000000  Default  30.0000  30.0000
978         flood  532.000000  532.000000  Default  29.0000  29.0000
980      flooding  403.000000  403.000000  Default  28.0000  28.0000
1903      project  736.000000  736.000000  Default  27.0000  27.0000
981        floods  442.000000  442.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
2651         work   91.063983  354.012281   Topic4  -5.5588   0.3959
672   development  107.989204  806.472334   Topic4  -5.3883  -0.2569
2658        world   99.185294  565.989693   Topic4  -5.4734   0.0121
1342         june   86.950542  410.081554   Topic4  -5.6050   0.2027
2207     services   80.414381  322.574375   Topic4  -5.6832   0.3646

[318 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
9         1  0.323169     access
9         2  0.009233     access
9         3  0.637104     access
9         4  0.027700     access
19        1  0.969505  achieving
...     ...       ...        ...
2669      4  0.097381       year
2670      1  0.352825      years
2670      2  0.202995      years
2670      3  0.275493      years
2670      4  0.169163      years

[656 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 4])

In [34]:
# The TFIDF representation model
# not as helpful as TF-- could be due to French? Not sure

# pyLDAvis.lda_model.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer)

## Repeat process for Kumasi

In [35]:
kumasi = pandas.read_excel("LexisUni_articles/processed/kumasi_articles.xlsx")
kumasi_docs_raw = kumasi['full_article'].tolist()
print(len(kumasi_docs_raw))

334


In [38]:
# Create the TF vector represetnation
kumasi_tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.5, 
                                min_df = 10)

kumasi_dtm_tf = kumasi_tf_vectorizer.fit_transform(kumasi_docs_raw)
print(kumasi_dtm_tf.shape)

(334, 2433)


In [ ]:
# TF IDF = measures term importance relative to the other documents
kumasi_tfidf_vectorizer = TfidfVectorizer(**kumasi_tf_vectorizer.get_params())

dtm_tfidf = tfidf_vectorizer.fit_transform(docs_raw)
print(dtm_tfidf.shape)